###1. Problems Encountered in the Map
Student response describes the challenges encountered while auditing, fixing and processing the dataset for the area of their choice. Some of the problems encountered during data audit are cleaned programmatically.


Student response shows understanding of the process of auditing, and ways to correct or standardize the data, including dealing with problems specific to the location, e.g. related to language or traditional ways of formatting. Some of the problems encountered during data audit are cleaned programmatically.

###2. Data Overview
Student provides a statistical overview about their chosen dataset, like:

    size of the file
    number of unique users
    number of nodes and ways
    number of chosen type of nodes, like cafes, shops etc
    
Student response provides the statistics about their chosen map area.

Student response also includes the MongoDB queries used to obtain the statistics.

###3. Additional Ideas
Other ideas about the datasets

Student is able to analyze the dataset and recognize opportunities for using it in other projects

Student proposes one or more additional ways of improving and analyzing the data and gives thoughtful discussion about the benefits and anticipated problems in implementing the improvement.

### Code Review:
####Code Functionality

All Lesson 6 problems are solved correctly. Final project code functionality reflects the description in the project document. All required Lesson 6 questions are correctly solved with the submitted code. Final project code functionality reflects the description in the project document.

####Code Readability

Final project code is well structured.

Final project code is commented as necessary.
Final project code follows an intuitive, easy-to-follow logical structure.

Final project code that is not intuitively readable is well-documented with comments.

###Thoroughness and Succinctness of Submission

Student submission is long enough to thoroughly answer the questions asked without giving unnecessary detail.
A good general guideline is that your question responses should take about 3-6 pages.